# **Downloading and unzipping the dataset**

In [ ]:
!wget https://bitbucket.org/ishaanjav/code-and-deploy-custom-tensorflow-lite-model/raw/a4febbfee178324b2083e322cdead7465d6fdf95/fruits.zip

--2023-01-27 12:04:34--  https://bitbucket.org/ishaanjav/code-and-deploy-custom-tensorflow-lite-model/raw/a4febbfee178324b2083e322cdead7465d6fdf95/fruits.zip
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105946856 (101M) [application/zip]
Saving to: ‘fruits.zip’

fruits.zip          100%[===================>] 101.04M  23.5MB/s    in 5.3s    

2023-01-27 12:04:44 (18.9 MB/s) - ‘fruits.zip’ saved [105946856/105946856]



In [ ]:
!unzip fruits.zip

Archive:  fruits.zip
   creating: fruits/
  inflating: __MACOSX/._fruits       
  inflating: fruits/.DS_Store        
  inflating: __MACOSX/fruits/._.DS_Store  
   creating: fruits/test/
   creating: fruits/train/
   creating: fruits/validation/
   creating: fruits/test/apple/
  inflating: fruits/test/.DS_Store   
  inflating: __MACOSX/fruits/test/._.DS_Store  
   creating: fruits/test/banana/
   creating: fruits/test/orange/
   creating: fruits/train/apple/
  inflating: fruits/train/.DS_Store  
  inflating: __MACOSX/fruits/train/._.DS_Store  
   creating: fruits/train/banana/
   creating: fruits/train/orange/
   creating: fruits/validation/apple/
  inflating: fruits/validation/.DS_Store  
  inflating: __MACOSX/fruits/validation/._.DS_Store  
   creating: fruits/validation/banana/
   creating: fruits/validation/orange/
  inflating: fruits/test/apple/Screen Shot 2018-06-08 at 5.27.19 PM.png  
  inflating: __MACOSX/fruits/test/apple/._Screen Shot 2018-06-08 at 5.27.19 PM.png  
  inflatin

# **Importing required libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# **Splitting the training, testing and validatinng data**

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/train",
    image_size=(32,32),
    batch_size=20
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/validation",
    image_size=(32,32),
    batch_size=20
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/test",
    image_size=(32,32),
    batch_size=20
)

#Training data is used to train the neural network and the validation data is used to check its score (accuracy) whereas
#testing data is the one where we test our neural network.

Found 460 files belonging to 3 classes.
Found 66 files belonging to 3 classes.
Found 130 files belonging to 3 classes.


# **Looking at our data**

In [ ]:
class_names = ["apple", "banana", "orange"]
for images, labels in train_ds.take(1):
  for i in range(5):
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.show()

# **Building the layers of our neural network**

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Rescaling(1./255),

     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),

     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),

     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128, activation="relu"),
     tf.keras.layers.Dense(3)
    ]
)

#Convolutional layer (Conv2D) applies filters to the image so we can gather data from it 
#We applie 32 filters and kept the kernel size to 3 by 3.
#Maxpooling layer down samples the size of feature maps making it easier for our neural network to understand
#Flatten layer turns the matrix into one dimensional array 

# **Compiling our neural network**

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

# **Fitting data and training our neural network**

In [ ]:
model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10
)

# **Evaluating our neural network**

In [ ]:
model.evaluate(test_ds)

7/7 [==============================] - 0s 12ms/step - loss: 0.1490 - accuracy: 0.9538


[0.14903397858142853, 0.9538461565971375]

# **Testing our neural network**

In [ ]:
import numpy

plt.figure(figsize=(10,10))
for images, labels in test_ds.take(1):
  # Predicts our test image
  classifications = model(images)
  
  for i in range(9):
    ax = plt.subplot(4, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))

    #Returns the highest value
    index = numpy.argmax(classifications[i])
    plt.title("Pred: " + class_names[index] + " | Real: " + class_names[labels[i]])